# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120255e+02     1.579727e+00
 * time: 0.13536596298217773
     1     1.114429e+01     9.213813e-01
 * time: 0.9808518886566162
     2    -1.232745e+01     1.095610e+00
 * time: 1.1190190315246582
     3    -3.386204e+01     8.936710e-01
 * time: 1.4183568954467773
     4    -4.742967e+01     6.447175e-01
 * time: 1.8536789417266846
     5    -5.680124e+01     2.146038e-01
 * time: 1.9975969791412354
     6    -5.964691e+01     1.694260e-01
 * time: 2.3076589107513428
     7    -6.084574e+01     7.411796e-02
 * time: 2.425581932067871
     8    -6.137020e+01     4.595670e-02
 * time: 2.5433850288391113
     9    -6.168367e+01     4.067040e-02
 * time: 2.6557459831237793
    10    -6.185960e+01     3.295004e-02
 * time: 2.772977113723755
    11    -6.201269e+01     2.336900e-02
 * time: 2.889857053756714
    12    -6.210748e+01     1.821666e-02
 * time: 3.006398916244507
    13    -6.216897e+01     1.366141e-02
 * time: 3.1232059001922

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671070
    AtomicLocal         -18.8557667
    AtomicNonlocal      14.8522634
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485370 
    Xc                  -19.3336820

    total               -62.261666461960
